In [1]:
# import library
from bs4 import BeautifulSoup
import requests
import json

In [4]:
# read prs
pr_path = 'Data/Data/merged feature pr with area labels with issues.json'

# read json file
with open(pr_path, 'r') as f:
    prs = json.load(f)

In [53]:
# get all the html urls
html_urls = []

for pr in prs:
    html_url = pr['html_url']
    html_urls.append(html_url)

In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the WebDriver (ensure you have the appropriate driver installed, e.g., chromedriver)
driver = webdriver.Chrome()
pr_issues_links = []

# Loop through the URLs, limited to the first 3 for this example
for url in html_urls:
    # Load the webpage
    driver.get(url)
    
    try:
        # Wait for the element to appear (max wait time: 10 seconds)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.Truncate-text.Link--primary.markdown-title.text-bold'))
        )
    except Exception as e:
        print(f"Element not found within the wait time for URL: {url}")
        continue

    # Get the page source
    page_source = driver.page_source

    # Parse with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Get <a> elements with the specified class
    a_tags = soup.find_all('a', class_='Truncate-text Link--primary markdown-title text-bold')

    # Get the href and associate with PR URL
    for a in a_tags:
        pr_issues_links.append({'issue_url': a['href'], 'pr_url': url})

# Close the WebDriver
driver.quit()

In [61]:
len(pr_issues_links)

750

In [60]:
# export the pr_issues_links
pr_issues_links_path = 'Data/Data/pr_issues_links_new.json'

with open(pr_issues_links_path, 'w') as f:
    json.dump(pr_issues_links, f)